# Homework 10

## Assignment 2: NLP 

### 2.1 Dataset


In [1]:
# open the txt file
bible = open("bible.txt", "r")  
# read file
data = bible.read()  

In [2]:
# test 
data[0:10]

'The First '

### 2.2 Word Embeddings

In [3]:
# useful imports 
import tensorflow as tf
from tensorflow_text import RegexSplitter
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, AveragePooling2D, TimeDistributed, LSTM, GlobalAvgPool2D, AbstractRNNCell, MaxPooling2D, RNN
import numpy as np
import matplotlib.pyplot as plt
import re

In [21]:
# global hyperparameters
VOCAB_SIZE = 10000
WINDOW_SIZE = 4

In [5]:
# convert to lower case and replace new-line characters with <space>
data = data.lower().replace("\n", " ")
# delete special characters, only alphanumeric values remain
# do we care for numbers or should we also delete them?
data = re.sub('\W+',' ', data)

In [6]:
# test 
data[0:100]

'the first book of moses called genesis 1 1 in the beginning god created the heaven and the earth 1 2'

In [7]:
# initalize tokenizer - splits on space
splitter = RegexSplitter(split_regex=' ')
# tokenize data
tokenized_data = splitter.split(data)

In [8]:
# test
tokenized_data[0:100]

<tf.RaggedTensor [[b'the', b'first', b'book', ..., b'you', b'all', b'amen']]>

In [9]:
# ragged tensor with tokens has weird shape, 
# so make it into normal tensor and reshape 
tokenized_data.get_shape()

TensorShape([1, None])

In [10]:
tokens = tokenized_data.to_tensor()
tokens.shape

TensorShape([1, 854033])

In [11]:
tokens = tf.reshape(tokens, (-1))
tokens.shape

TensorShape([854033])

In [12]:
# get unique words in the text with their counts 
# such that we can later use them to create vocabulary with most common words

'''
unique_with_counts returns:
unique_tokens = tensor with unique tokens in the order they occur in tokens
idx = same size as tokens; for each token in the original tokens it shows the index with regards to the new unique_tokens tensor
count = contains the count of each element of unique_tokens in tokens
''' 

unique_tokens, idx, counts = tf.unique_with_counts(tokens)

In [13]:
#test
unique_tokens[0:100]

<tf.Tensor: shape=(100,), dtype=string, numpy=
array([b'the', b'first', b'book', b'of', b'moses', b'called', b'genesis',
       b'1', b'in', b'beginning', b'god', b'created', b'heaven', b'and',
       b'earth', b'2', b'was', b'without', b'form', b'void', b'darkness',
       b'upon', b'face', b'deep', b'spirit', b'moved', b'waters', b'3',
       b'said', b'let', b'there', b'be', b'light', b'4', b'saw', b'that',
       b'it', b'good', b'divided', b'from', b'5', b'day', b'he', b'night',
       b'evening', b'morning', b'were', b'6', b'a', b'firmament',
       b'midst', b'divide', b'7', b'made', b'which', b'under', b'above',
       b'so', b'8', b'second', b'9', b'gathered', b'together', b'unto',
       b'one', b'place', b'dry', b'land', b'appear', b'10', b'gathering',
       b'seas', b'11', b'bring', b'forth', b'grass', b'herb', b'yielding',
       b'seed', b'fruit', b'tree', b'after', b'his', b'kind', b'whose',
       b'is', b'itself', b'12', b'brought', b'13', b'third', b'14',
       b'li

In [14]:
# gives indices of sorted counts
sorted_counts = tf.argsort(counts ,axis=-1,direction='DESCENDING',stable=False)

In [15]:
# test
# sorted_counts[0:10000]

In [16]:
# get list of the most common 10 000 words in the bible
# by using the indices of the most counts and applying them to the unique_tokens
most_common_words = list()

for i in sorted_counts[0:VOCAB_SIZE]:
    most_common_words.append(unique_tokens[i])

In [28]:
# create new dataset out of the most common words
vocab = tf.data.Dataset.from_tensors(most_common_words)

In [29]:
# test
list(vocab.as_numpy_iterator())

[array([b'the', b'and', b'of', ..., b'ashbea', b'jokim', b'chozeba'],
       dtype=object)]

In [22]:
# to create input-target pairs (= word-contexts pairs) we use 
# tf's continuous skipgram model 
# https://www.tensorflow.org/tutorials/text/word2vec


bible_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      tokens,
      vocabulary_size=VOCAB_SIZE,
      window_size=WINDOW_SIZE,
      negative_samples=0)



KeyboardInterrupt: 

In [40]:
for target, context in bible_skip_grams[:5]:
    print(target, context) 

NameError: name 'bible_skip_grams' is not defined